### Задание № 1 (4 балла)

Попробуйте матричные разложения с 5 классификаторами - SGDClassifier, KNeighborsClassifier, RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF и SVD. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 10 моделей (два разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF, иначе точно будет слишком долго)

In [3]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 8.2 MB 10.5 MB/s 


In [5]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 11.3 MB 19.5 MB/s 
     |████████████████████████████████| 15.7 MB 8.3 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=187bb5e569f9c2ecb98330b917204e92d50838e677cfd1059a9cee826cb33c19
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [2]:
!pip install razdel

In [3]:
import gensim
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim_models
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
morph = MorphAnalyzer()
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

In [4]:
# добавим лемматизацию
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

In [ ]:
data = pd.read_csv('avito_category_classification.csv')
data['description_norm'] = data['description'].apply(normalize)

In [ ]:
data

,category_name,description,description_norm
0,Автомобили,"отличное состояние,обслужиание в салоне",отличный состояние обслужиание в салон
1,Детская одежда и обувь,В отличном состоянии. Фирма KIKO. Очень теплый...,в отличный состояние фирма kiko очень тёплый у...
2,Предложение услуг,"Изготовление ограждений, перил,качелей, турник...",изготовление ограждение перила качели турников...
3,Автомобили,Автомобиль в отличном техническом состоянии. О...,автомобиль в отличный технический состояние од...
4,Бытовая техника,"Продается газовая плита ""Гефест"" (Белоруссия) ...",продаваться газовый плита гефест белоруссия б ...
...,...,...,...
9893,Товары для детей и игрушки,Чтобы посмотреть весь ассортимент нашего магаз...,чтобы посмотреть весь ассортимент наш магазин ...
9894,Детская одежда и обувь,"Весна,осень.74-80.вопросы можно в вайбер,двухс...",весна осень 74-80 вопрос можно в вайбер двухст...
9895,"Одежда, обувь, аксессуары","Кимоно Green Hill. Состояние отличное, рост ...",кимоно green hill состояние отличный рост 160-...
9896,Детская одежда и обувь,Б/у кроссовки на девочку. Носили только в спор...,б у кроссовок на девочка носить только в спортзал


In [ ]:
# classification_report с встроенной кросс-валидацией

def eval_table(X, y, pipeline, N=6):
    labels = list(set(y))
    fold_metrics = pd.DataFrame(index=labels)
    errors = np.zeros((len(labels), len(labels)))
    
    kfold = StratifiedKFold(n_splits=N, shuffle=True, )

    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])
        
        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')
    
    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result.loc['mean'] = result.mean().round(2)
    errors /= N
    
    return result, errors

RandomForestClassifier

In [ ]:
pipeline_svd_rfc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))
])

pipeline_nmf_rfc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(100)),
    ('clf', RandomForestClassifier(n_estimators=200, max_depth=6))
])

In [ ]:
metrics_svd_rfc, errors_svd_rfc = eval_table(data['description_norm'], data['category_name'], pipeline_svd_rfc)
metrics_nmf_rfc, errors_nmf_rfc = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_rfc)

In [ ]:
metrics_svd_rfc

,precision,precision_std,recall,recall_std,f1,f1_std
Предложение услуг,0.74,0.09,0.50,0.04,0.60,0.04
Ремонт и строительство,0.71,0.15,0.09,0.04,0.15,0.06
Товары для детей и игрушки,0.82,0.07,0.18,0.04,0.29,0.06
"Одежда, обувь, аксессуары",0.47,0.01,0.77,0.02,0.58,0.01
Бытовая техника,0.90,0.23,0.05,0.02,0.10,0.04
Детская одежда и обувь,0.45,0.01,0.68,0.01,0.54,0.01
Квартиры,0.88,0.04,0.88,0.02,0.88,0.01
Мебель и интерьер,0.83,0.26,0.03,0.02,0.06,0.03
Автомобили,0.86,0.04,0.59,0.08,0.70,0.06
Телефоны,0.96,0.04,0.33,0.03,0.49,0.03


In [ ]:
metrics_nmf_rfc

,precision,precision_std,recall,recall_std,f1,f1_std
Предложение услуг,0.72,0.07,0.49,0.08,0.58,0.06
Ремонт и строительство,0.75,0.17,0.06,0.03,0.11,0.06
Товары для детей и игрушки,0.88,0.05,0.33,0.04,0.48,0.05
"Одежда, обувь, аксессуары",0.44,0.02,0.83,0.04,0.57,0.02
Бытовая техника,0.67,0.52,0.02,0.02,0.04,0.04
Детская одежда и обувь,0.64,0.03,0.70,0.05,0.66,0.02
Квартиры,0.87,0.03,0.96,0.03,0.91,0.02
Мебель и интерьер,0.75,0.13,0.12,0.08,0.19,0.11
Автомобили,0.89,0.04,0.77,0.04,0.82,0.02
Телефоны,0.90,0.04,0.31,0.04,0.46,0.04


SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
pipeline_svd_sgd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', SGDClassifier())
])

pipeline_nmf_sgd = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(100)),
    ('clf', SGDClassifier())
])

In [ ]:
metrics_svd_sgd, errors_svd_sgd = eval_table(data['description_norm'], data['category_name'], pipeline_svd_sgd)
metrics_nmf_sgd, errors_nmf_sgd = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_sgd)

In [ ]:
metrics_svd_sgd

,precision,precision_std,recall,recall_std,f1,f1_std
Предложение услуг,0.77,0.02,0.74,0.05,0.76,0.02
Ремонт и строительство,0.55,0.03,0.54,0.07,0.54,0.03
Товары для детей и игрушки,0.73,0.03,0.66,0.03,0.69,0.02
"Одежда, обувь, аксессуары",0.73,0.03,0.75,0.03,0.74,0.01
Бытовая техника,0.66,0.10,0.49,0.08,0.56,0.08
Детская одежда и обувь,0.74,0.03,0.79,0.03,0.76,0.01
Квартиры,0.96,0.02,0.96,0.02,0.96,0.01
Мебель и интерьер,0.70,0.07,0.63,0.04,0.66,0.03
Автомобили,0.87,0.03,0.90,0.02,0.88,0.01
Телефоны,0.81,0.06,0.79,0.05,0.80,0.04


In [ ]:
metrics_nmf_sgd

,precision,precision_std,recall,recall_std,f1,f1_std
Предложение услуг,0.62,0.15,0.43,0.20,0.46,0.14
Ремонт и строительство,0.30,0.20,0.22,0.27,0.17,0.14
Товары для детей и игрушки,0.79,0.08,0.39,0.09,0.51,0.07
"Одежда, обувь, аксессуары",0.62,0.09,0.68,0.14,0.63,0.02
Бытовая техника,0.37,0.22,0.08,0.06,0.13,0.07
Детская одежда и обувь,0.63,0.08,0.67,0.09,0.64,0.02
Квартиры,0.81,0.09,0.96,0.02,0.88,0.05
Мебель и интерьер,0.57,0.15,0.40,0.11,0.45,0.08
Автомобили,0.72,0.16,0.61,0.22,0.62,0.09
Телефоны,0.62,0.25,0.62,0.23,0.54,0.15


KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
pipeline_svd_knc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', KNeighborsClassifier())
])

pipeline_nmf_knc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(100)),
    ('clf', KNeighborsClassifier())
])

In [ ]:
metrics_svd_knc, errors_svd_knc = eval_table(data['description_norm'], data['category_name'], pipeline_svd_knc)
metrics_nmf_knc, errors_nmf_knc = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_knc)

In [ ]:
metrics_svd_knc

,precision,precision_std,recall,recall_std,f1,f1_std
Предложение услуг,0.58,0.04,0.58,0.04,0.58,0.03
Ремонт и строительство,0.26,0.08,0.16,0.06,0.20,0.06
Товары для детей и игрушки,0.54,0.06,0.25,0.03,0.34,0.04
"Одежда, обувь, аксессуары",0.47,0.01,0.58,0.03,0.52,0.02
Бытовая техника,0.27,0.06,0.27,0.07,0.27,0.06
Детская одежда и обувь,0.48,0.01,0.62,0.02,0.54,0.01
Квартиры,0.94,0.01,0.71,0.04,0.81,0.03
Мебель и интерьер,0.36,0.06,0.28,0.05,0.31,0.05
Автомобили,0.53,0.04,0.59,0.04,0.56,0.04
Телефоны,0.68,0.05,0.32,0.03,0.44,0.03


In [ ]:
metrics_nmf_knc

,precision,precision_std,recall,recall_std,f1,f1_std
Предложение услуг,0.45,0.05,0.68,0.05,0.54,0.05
Ремонт и строительство,0.27,0.03,0.24,0.03,0.25,0.02
Товары для детей и игрушки,0.51,0.06,0.35,0.04,0.42,0.04
"Одежда, обувь, аксессуары",0.54,0.02,0.54,0.05,0.54,0.03
Бытовая техника,0.20,0.03,0.28,0.05,0.23,0.03
Детская одежда и обувь,0.54,0.03,0.53,0.03,0.53,0.02
Квартиры,0.90,0.04,0.84,0.04,0.87,0.03
Мебель и интерьер,0.42,0.09,0.32,0.06,0.36,0.07
Автомобили,0.52,0.04,0.72,0.03,0.60,0.03
Телефоны,0.65,0.07,0.45,0.08,0.53,0.06


ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
pipeline_svd_etc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
    ('svd', TruncatedSVD(500)),
    ('clf', ExtraTreesClassifier())
])

pipeline_nmf_etc = Pipeline([
    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), min_df=3, max_df=0.3)),
    ('tfidf', TfidfTransformer()),
    ('decomposition', NMF(100)),
    ('clf', ExtraTreesClassifier())
])

In [ ]:
metrics_svd_etc, errors_svd_etc = eval_table(data['description_norm'], data['category_name'], pipeline_svd_etc)
metrics_nmf_etc, errors_nmf_etc = eval_table(data['description_norm'], data['category_name'], pipeline_nmf_etc)

In [ ]:
metrics_svd_etc

,precision,precision_std,recall,recall_std,f1,f1_std
Предложение услуг,0.86,0.06,0.27,0.04,0.42,0.06
Ремонт и строительство,0.59,0.11,0.08,0.03,0.14,0.05
Товары для детей и игрушки,0.67,0.09,0.16,0.02,0.26,0.03
"Одежда, обувь, аксессуары",0.46,0.01,0.71,0.02,0.56,0.01
Бытовая техника,0.55,0.16,0.14,0.05,0.22,0.07
Детская одежда и обувь,0.44,0.02,0.72,0.02,0.55,0.02
Квартиры,0.71,0.02,0.77,0.04,0.74,0.02
Мебель и интерьер,0.80,0.13,0.13,0.04,0.23,0.06
Автомобили,0.81,0.05,0.42,0.03,0.55,0.03
Телефоны,0.90,0.06,0.32,0.05,0.47,0.05


In [ ]:
metrics_nmf_etc

,precision,precision_std,recall,recall_std,f1,f1_std
Предложение услуг,0.69,0.05,0.74,0.05,0.71,0.04
Ремонт и строительство,0.59,0.04,0.43,0.06,0.50,0.05
Товары для детей и игрушки,0.73,0.04,0.59,0.03,0.65,0.03
"Одежда, обувь, аксессуары",0.70,0.01,0.79,0.02,0.74,0.01
Бытовая техника,0.66,0.06,0.33,0.07,0.44,0.07
Детская одежда и обувь,0.73,0.02,0.77,0.02,0.75,0.01
Квартиры,0.91,0.03,0.97,0.03,0.94,0.02
Мебель и интерьер,0.72,0.03,0.54,0.02,0.62,0.02
Автомобили,0.84,0.04,0.91,0.01,0.87,0.03
Телефоны,0.81,0.05,0.79,0.04,0.80,0.04


Сравнение:

In [ ]:
df = pd.DataFrame({'Matrix': ['SVD', 'NMF'], 'RFC': [0.44, 0.48], 'SGD': [0.74, 0.5], 'KNC': [0.46, 0.49], 'ETC': [0.41, 0.70]}, )
df

,Matrix,RFC,SGD,KNC,ETC
0,SVD,0.44,0.74,0.46,0.41
1,NMF,0.48,0.50,0.49,0.70


Таким образом, наилучшие показатели у модели SGD-SVD.

### Задание № 2 (6 баллов)

В Gensim тоже можно добавить нграммы и tfidf. Постройте 1 модель без них (как в семинаре) и еще 3 модели (1 с нграммами, 1 с tfidf и 1 с нграммами и с tfidf). Сравните качество с помощью метрик (перплексия, когерентность) и на глаз. Определите лучшую модель. Для каждой модели выберите 1 самую красивую на ваш взгляд тему.

Используйте данные википедии из семинара. Можете взять поменьше данных, если все обучается долго.

Важное требование - получившиеся модели не должны быть совсем плохими. Если хороших тем не получается, попробуйте настроить гиперпараметры, отфильтровать словарь по-другому. 

ПРОСТО LDA КАК НА СЕМИНАРЕ:

In [5]:
texts = open('wiki_data.txt').read().splitlines()[:1000] # взяла меньше данных
texts = ([normalize(text) for text in texts])
# texts [:3]

In [ ]:
dictionary = gensim.corpora.Dictionary((text.split() for text in texts))

In [ ]:
print(dictionary)

In [ ]:
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

In [ ]:
corpus = [dictionary.doc2bow(text.split()) for text in texts]

In [ ]:
# corpus [:3] #. почему тут только числа?

In [ ]:
lda = gensim.models.LdaMulticore(corpus, 
                                 100, # количество тем
                                 alpha='asymmetric',
                                 id2word=dictionary, 
                                 passes=10)
lda.print_topics()

[(99,
  '0.077*"штат" + 0.021*"конституция" + 0.019*"представитель" + 0.013*"борьба" + 0.013*"служба" + 0.012*"конгресс" + 0.012*"план" + 0.011*"поэтому" + 0.011*"государство" + 0.010*"участник"'),
 (98,
  '0.043*"религиозный" + 0.033*"передача" + 0.029*"здание" + 0.024*"церковь" + 0.023*"организация" + 0.022*"закон" + 0.020*"2010" + 0.019*"культура" + 0.017*"собственность" + 0.017*"объект"'),
 (97,
  '0.080*"уезд" + 0.022*"городской" + 0.022*"специальный" + 0.019*"икона" + 0.019*"доска" + 0.019*"кнр" + 0.017*"расформировать" + 0.016*"округ" + 0.016*"преобразовать" + 0.016*"власть"'),
 (96,
  '0.026*"атлетика" + 0.025*"спортсмен" + 0.025*"1984" + 0.024*"фиджи" + 0.017*"1948" + 0.017*"парусный" + 0.016*"спорт" + 0.015*"1940" + 0.015*"дзюдо" + 0.014*"тяжёлый"'),
 (95,
  '0.090*"соревнование" + 0.088*"атлетика" + 0.080*"спортсмен" + 0.070*"лёгкий" + 0.055*"2004" + 0.039*"греция" + 0.035*"2000" + 0.033*"тяжёлый" + 0.023*"афины" + 0.019*"плавание"'),
 (94,
  '0.043*"1976" + 0.038*"канада" +

Вывод: этот способ даёт нормальное разбиение по темам, в принципе все они так или иначе хорошо считываются. Возможно, самая удачная из представленных - 5:
(5,
  '0.077*"клуб" + 0.061*"команда" + 0.053*"чемпионат" + 0.022*"сезон" + 0.020*"матч" + 0.020*"лига" + 0.019*"футбольный" + 0.017*"кубок" + 0.017*"барселона" + 0.016*"футбол"')

In [ ]:
np.exp2(-lda.log_perplexity(corpus[:1000]))

215.30352901689452

In [ ]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text.split() for text in texts], 
                                                   dictionary=dictionary, coherence='c_v')

coherence_model_lda.get_coherence()

0.4820069670437174

 LDA С N-ГРАММАМИ:

In [1]:
texts = open('wiki_data.txt').read().splitlines()[:1000]
texts = ([normalize(text) for text in texts])
texts [:3]

In [6]:
texts_n = [text.split() for text in texts]
ph = gensim.models.Phrases(texts_n, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts_n] 

In [7]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [8]:
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

In [9]:
corpus = [dictionary.doc2bow(ngram) for ngram in ngrammed_texts]

In [10]:
lda = gensim.models.LdaMulticore(corpus, 
                                 100, # количество тем
                                 alpha='asymmetric',
                                 id2word=dictionary, 
                                 passes=10)
lda.print_topics()

[(99,
  '0.026*"монастырь" + 0.017*"собор" + 0.013*"я" + 0.009*"башня" + 0.009*"святой" + 0.008*"появиться" + 0.008*"умереть" + 0.007*"сша" + 0.007*"фильм" + 0.007*"фон"'),
 (98,
  '0.031*"сельский_поселение" + 0.027*"поселение" + 0.022*"округ" + 0.019*"община" + 0.018*"контакт" + 0.014*"№" + 0.014*"закон" + 0.013*"граница" + 0.013*"сообщение" + 0.012*"территория"'),
 (97,
  '0.017*"сезон" + 0.013*"лига" + 0.012*"система" + 0.010*"иран" + 0.009*"участвовать" + 0.008*"высокий" + 0.008*"залив" + 0.008*"кубок" + 0.007*"победить" + 0.007*"убить"'),
 (96,
  '0.062*"1-й" + 0.054*"1978" + 0.043*"1976" + 0.038*"1977" + 0.036*"1984" + 0.030*"участник" + 0.028*"игра_1984" + 0.027*"1975" + 0.026*"турнир" + 0.026*"4-й"'),
 (95,
  '0.051*"дом" + 0.021*"день" + 0.016*"здание" + 0.014*"музей" + 0.011*"проект" + 0.010*"совместно_с" + 0.010*"семья" + 0.009*"состояться" + 0.009*"н" + 0.009*"праздник"'),
 (94,
  '0.023*"вода" + 0.022*"доска" + 0.020*"озеро" + 0.016*"верхний" + 0.015*"деревянный" + 0.011*

Вывод: этот способ также даёт нормальное разбиение по темам. Хорошая тема также получилась про спорт:
(2,
  '0.023*"игра" + 0.021*"сезон" + 0.018*"вид" + 0.017*"серия" + 0.015*"команда" + 0.015*"очко" + 0.012*"боливия" + 0.012*"боливия_принимать" + 0.012*"играть" + 0.012*"счёт"')

In [11]:
np.exp2(-lda.log_perplexity(corpus[:1000]))

257.8143177841666

In [12]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictionary, coherence='c_v')

coherence_model_lda.get_coherence()

0.47233490085449886

МОДЕЛЬ С TF IDF:

In [3]:
texts = open('wiki_data.txt').read().splitlines()[:1000]
texts = ([normalize(text) for text in texts])
# texts [:3]

In [ ]:
dictionary = gensim.corpora.Dictionary((text.split() for text in texts))

In [ ]:
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

In [ ]:
corpus = [dictionary.doc2bow(text.split()) for text in texts]

In [ ]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary, )
corpus = tfidf[corpus]

In [ ]:
lda = gensim.models.LdaMulticore(corpus, 
                                 100, # количество тем
                                 alpha='asymmetric',
                                 id2word=dictionary, 
                                 passes=10)
lda.print_topics()

[(99,
  '0.000*"конечный" + 0.000*"круглый" + 0.000*"изобразить" + 0.000*"историк" + 0.000*"каменный" + 0.000*"комиссар" + 0.000*"конгресс" + 0.000*"здание" + 0.000*"красивый" + 0.000*"лишить"'),
 (98,
  '0.001*"сельсовет" + 0.001*"альбом" + 0.000*"душа" + 0.000*"сеть" + 0.000*"мы" + 0.000*"хутор" + 0.000*"проживать" + 0.000*"стена" + 0.000*"сотня" + 0.000*"музыкант"'),
 (97,
  '0.001*"сценарий" + 0.000*"фамилия" + 0.000*"окончить" + 0.000*"москва" + 0.000*"1953" + 0.000*"факультет" + 0.000*"руководитель" + 0.000*"считать" + 0.000*"2008" + 0.000*"8"'),
 (95,
  '0.001*"институт" + 0.001*"научный" + 0.001*"учёный" + 0.001*"университет" + 0.001*"директор" + 0.001*"солнце" + 0.000*"московский" + 0.000*"русский" + 0.000*"заведовать" + 0.000*"ликвидация"'),
 (96,
  '0.000*"конечный" + 0.000*"круглый" + 0.000*"изобразить" + 0.000*"историк" + 0.000*"каменный" + 0.000*"комиссар" + 0.000*"конгресс" + 0.000*"здание" + 0.000*"красивый" + 0.000*"лишить"'),
 (94,
  '0.000*"военно-морской" + 0.000*"о

Вывод: визуально этот способ также даёт нормальное разбиение по темам. Метрики сравним в конце исследования. Неплохая тема получилась про политику:
(91,
  '0.001*"выборы" + 0.001*"дума" + 0.001*"георгий" + 0.001*"александрович" + 0.001*"выдвинуть" + 0.001*"тур" + 0.001*"партия" + 0.001*"депутат" + 0.001*"кандидат" + 0.000*"городской"')

In [ ]:
np.exp2(-lda.log_perplexity(corpus[:1000]))

1759.4617775119436

In [7]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text.split() for text in texts], 
                                                   dictionary=dictionary, coherence='c_v')

coherence_model_lda.get_coherence()

С N-GRAMS И TF IDF:

In [5]:
texts = open('wiki_data.txt').read().splitlines()[:1000]
texts = ([normalize(text) for text in texts])
# texts [:3]

In [4]:
# print(texts[0].split())

In [18]:
texts_n = [text.split() for text in texts]
ph = gensim.models.Phrases(texts_n, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts_n] 

In [19]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [6]:
# for i in range (3):
#   print (ngrammed_texts[i])

In [20]:
dictionary.filter_extremes(no_above=0.1, no_below=10)
dictionary.compactify()

In [21]:
corpus = [dictionary.doc2bow(ngram) for ngram in ngrammed_texts]

In [22]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary, )
corpus = tfidf[corpus]

In [23]:
lda = gensim.models.LdaMulticore(corpus, 
                                 100, # количество тем
                                 alpha='asymmetric',
                                 id2word=dictionary, 
                                 passes=10)
lda.print_topics()

[(99,
  '0.001*"я" + 0.000*"деньга" + 0.000*"ученик" + 0.000*"художник" + 0.000*"чтобы" + 0.000*"уехать" + 0.000*"взять" + 0.000*"сделать" + 0.000*"дом" + 0.000*"настоящий"'),
 (97,
  '0.000*"священник" + 0.000*"символ" + 0.000*"религиозный" + 0.000*"решение" + 0.000*"рим" + 0.000*"рядом" + 0.000*"самостоятельный" + 0.000*"свой_очередь" + 0.000*"расположение" + 0.000*"северный_часть"'),
 (98,
  '0.000*"священник" + 0.000*"символ" + 0.000*"религиозный" + 0.000*"решение" + 0.000*"рим" + 0.000*"рядом" + 0.000*"самостоятельный" + 0.000*"свой_очередь" + 0.000*"расположение" + 0.000*"северный_часть"'),
 (96,
  '0.000*"священник" + 0.000*"символ" + 0.000*"религиозный" + 0.000*"решение" + 0.000*"рим" + 0.000*"рядом" + 0.000*"самостоятельный" + 0.000*"свой_очередь" + 0.000*"расположение" + 0.000*"северный_часть"'),
 (95,
  '0.000*"священник" + 0.000*"символ" + 0.000*"религиозный" + 0.000*"решение" + 0.000*"рим" + 0.000*"рядом" + 0.000*"самостоятельный" + 0.000*"свой_очередь" + 0.000*"расположен

Вывод: визуально этот способ хуже остальных, или я что-то неправильно сделала. Метрики сравним в конце исследования. Неплохая тема получилась про религию, но не понимаю, почему одна тема повторяется несколько раз:
 (94,
  '0.000*"священник" + 0.000*"символ" + 0.000*"религиозный" + 0.000*"решение" + 0.000*"рим" + 0.000*"рядом" + 0.000*"самостоятельный" + 0.000*"свой_очередь" + 0.000*"расположение" + 0.000*"северный_часть"')

In [15]:
np.exp2(-lda.log_perplexity(corpus[:1000]))

1775.725933761205

In [16]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictionary, coherence='c_v')

coherence_model_lda.get_coherence()

0.40068210222785716

Сравнение моделей:

In [24]:
df = pd.DataFrame({'Метрики': ['Перплексия', 'Когерентность'], 'Просто Gensim': [215.30, 0.48], 'N-grams': [257.84, 0.47], 'TfIdf': [1759.46, 0.44], 'N-grams+TfIdf': [1775.73, 0.40]}, )
df

,Метрики,Просто Gensim,N-grams,TfIdf,N-grams+TfIdf
0,Перплексия,215.30,257.84,1759.46,1775.73
1,Когерентность,0.48,0.47,0.44,0.40


По значениям перплексии мы видим, что лучше всего работает просто первоначальная модель (перплексия ближе всего к нулю). По показателям когерентности также делаем вывод, что лучше всего работает первоначальная модель (у неё максимальное значение когерентности).